<a href="https://colab.research.google.com/github/abhay-053/MentalHealthAnalysis/blob/main/FineTuningBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import DataLoader
from imblearn.over_sampling import RandomOverSampler
import os


from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report
from transformers import Trainer , TrainingArguments , BertTokenizer , BertForSequenceClassification



In [2]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
import re
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

from transformers import AutoModelForSequenceClassification , AutoTokenizer
os.environ["WANDB_MODE"] = "offline"

In [4]:
import pandas as pd

df = pd.read_csv("/content/Combined Data.csv", on_bad_lines='skip')
print("Successfully loaded with skipped bad lines.")


Successfully loaded with skipped bad lines.


In [5]:
print(df.columns)

Index(['Unnamed: 0', 'statement', 'status'], dtype='object')


In [6]:
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
40409,40409,it s almost funny i don t want you to feel lik...,Depression
40410,40410,for context i m a yr old mom of two kid my dau...,Depression
40411,40411,i feel like wanting to die why do i not have f...,Depression
40412,40412,my imagination is not vivid enough to have fai...,Depression


In [7]:
df = df.dropna()

In [8]:
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
40408,40408,i just don t care anymore i don t care what ha...,Depression
40409,40409,it s almost funny i don t want you to feel lik...,Depression
40410,40410,for context i m a yr old mom of two kid my dau...,Depression
40411,40411,i feel like wanting to die why do i not have f...,Depression


In [9]:
import torch
print("Using GPU?" , torch.cuda.is_available())


Using GPU? False


In [10]:
pd.DataFrame(df["status"].value_counts())

,count
status,
Depression,13121
Normal,12443
Suicidal,10652
Anxiety,2697
Stress,1488


In [11]:
df = df.sample(n=6000 , random_state=42).reset_index(drop=True)
df

,Unnamed: 0,statement,status
0,19903,"I give up looking it up on the internet, if an...",Suicidal
1,1997,"Wes doesn't care, we have to update the news o...",Normal
2,16377,I have depression since middleschool no friend...,Depression
3,12306,when will it end oh my god i cannot do it idk ...,Suicidal
4,15363,"Also just throwing it out there, you have very...",Depression
...,...,...,...
5995,22484,I am 17 yo and i really do not know why i feel...,Depression
5996,37009,now time for mouse to mouse pad for a long lon...,Normal
5997,36371,"@therealnph yeah, cool bookshelves I've got th...",Normal
5998,3667,"I'm not in the mood, I feel like I'm angry",Normal


In [12]:
df.shape

(6000, 3)

In [13]:
df = df.drop(columns=["Unnamed: 0"] , axis=1)
df

,statement,status
0,"I give up looking it up on the internet, if an...",Suicidal
1,"Wes doesn't care, we have to update the news o...",Normal
2,I have depression since middleschool no friend...,Depression
3,when will it end oh my god i cannot do it idk ...,Suicidal
4,"Also just throwing it out there, you have very...",Depression
...,...,...
5995,I am 17 yo and i really do not know why i feel...,Depression
5996,now time for mouse to mouse pad for a long lon...,Normal
5997,"@therealnph yeah, cool bookshelves I've got th...",Normal
5998,"I'm not in the mood, I feel like I'm angry",Normal


In [14]:
df.describe(include="all").T


,count,unique,top,freq
statement,6000,5980,thank you.,3
status,6000,5,Depression,1934


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  6000 non-null   object
 1   status     6000 non-null   object
dtypes: object(2)
memory usage: 93.9+ KB


In [16]:
pd.DataFrame(df["status"].value_counts())

,count
status,
Depression,1934
Normal,1895
Suicidal,1541
Anxiety,408
Stress,222


In [17]:
def preprocessing(text) :
    text = text.lower()
    text = re.sub(r"[^\w\s]" , "" , text)
    text = re.sub(r"\d+" , "" , text)
    words = text.split()
    words = [w for w in words if w not in stopwords]
    preprocessing = " ".join(words)
    return preprocessing

In [18]:
df["cleaned_text"] = df["statement"].apply(preprocessing)

df["statement_length"] = df["statement"].apply(len)

df["cleaned_text_length"] = df["cleaned_text"].apply(len)

In [19]:
df.head(10)

,statement,status,cleaned_text,statement_length,cleaned_text_length
0,"I give up looking it up on the internet, if an...",Suicidal,give looking internet know would greatly appre...,135,81
1,"Wes doesn't care, we have to update the news o...",Normal,wes doesnt care update news covid wkwkwk reall...,103,62
2,I have depression since middleschool no friend...,Depression,depression since middleschool friends nothing ...,1075,615
3,when will it end oh my god i cannot do it idk ...,Suicidal,end oh god cannot idk much longer take,77,38
4,"Also just throwing it out there, you have very...",Depression,also throwing nice eyes pls drink something to...,216,146
5,"We talked about meeting, and eventually arrang...",Normal,talked meeting eventually arranged double date...,389,249
6,"Iqbal, who is handsome, has a lot of money, an...",Normal,iqbal handsome lot money loyal zidny unclean,74,44
7,"Hello, thank you for reading this, first of al...",Depression,hello thank reading first alli wonder anyone f...,1381,800
8,people always tell me why i do not show any ha...,Depression,people always tell show happiness emotion acco...,830,516
9,teuask~ does anyone still sell the album ch1-2...,Normal,teuask anyone still sell album ch seal st press,73,47


In [20]:
df["cleaned_text"][10]


'lot therapists one used ask every week suicidal ideation would always say even though almost always liei ever really come close killing closest ever come pondering logistics killing ever decided eg method location etc gone store buy rope bought firearms anything like intend vast majority time even mostly really want alive anymore hurts muchbut guess sure upside telling therapist point talk stuff bothering anyway add anything conversationas far downsides involuntarily committed mental institution one worst fears understanding involuntarily committed pose danger others think also worry telling therapist thinking suicide take determination hands would rather therapist ask question guy pose risk othersi live pennsylvania matters point telling therapist experience suicidal ideation'

In [21]:
df['status'].value_counts()


,count
status,
Depression,1934
Normal,1895
Suicidal,1541
Anxiety,408
Stress,222


In [22]:
# Split
X = df.drop(columns=["status"] , axis=1)
y = df["status"]

In [23]:
Ra_ov_sam = RandomOverSampler(random_state=42)
X_train_res, y_train_res = Ra_ov_sam.fit_resample(X, y)

df = pd.concat([X_train_res, y_train_res] , axis=1)

In [24]:
pd.DataFrame(df['status'].value_counts())


,count
status,
Suicidal,1934
Normal,1934
Depression,1934
Stress,1934
Anxiety,1934


In [25]:
encode = LabelEncoder()

df["status"] = encode.fit_transform(df["status"])


X = df["statement"]

y = df["status"]

In [26]:
num_labels = len(df["status"].unique())
num_labels

5

In [27]:
X_train , X_test , y_train , y_test = train_test_split(X , y , test_size=0.2 , random_state=42)


print(f"X Train Shape is = {X_train.shape}")
print(f"X Test Shape is = {X_test.shape}")
print(f"y Train Shape is = {y_train.shape}")
print(f"y Test Shape is = {y_test.shape}")

X Train Shape is = (7736,)
X Test Shape is = (1934,)
y Train Shape is = (7736,)
y Test Shape is = (1934,)


In [28]:
y_train.unique()

array([4, 2, 3, 1, 0])

In [29]:
max([len(c) for c in df["statement"]])

13054

In [30]:
# Tokenization using BERT tokenizer

tok = BertTokenizer.from_pretrained("bert-base-uncased")

train_encoding = tok(list(X_train) , padding=True , truncation=True , max_length=200)
test_encodeing = tok(list(X_test) , padding=True , truncation=True , max_length=200)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [31]:
# Convert to Dataset format

from datasets import Dataset

df_train = Dataset.from_dict({"input_ids" : train_encoding["input_ids"] ,
                              "attention_mask" : train_encoding["attention_mask"] ,
                              "labels" : y_train.tolist()})
df_test = Dataset.from_dict({"input_ids" : test_encodeing["input_ids"] ,
                             "attention_mask" : test_encodeing["attention_mask"] ,
                             "labels" : y_test.tolist()})

In [32]:
import transformers
print(transformers.__version__)

4.51.3


In [35]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased" , num_labels=num_labels)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    lr_scheduler_type="linear",
    warmup_steps=500,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=3,
    gradient_accumulation_steps=2 ,
    report_to="none"  # Disables WandB logging
)


# Trainer
trainer = Trainer(
    model=model ,
    args=training_args ,
    train_dataset=df_train ,
    eval_dataset=df_test
)


# Fine-tuning the model

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
pred , label , _ = trainer.predict(df_test)

pred_labels = np.argmax(pred , axis=1)


In [ ]:
cm = confusion_matrix(pred_labels , y_test)
cm


In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(cm , annot=True , cbar=True , cmap="Blues" , xticklabels=num_labels , yticklabels=num_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
print(classification_report(pred_labels , y_test , target_names=encode.classes_))

In [ ]:
trainer.save_model("saved_mental_status_bert")

tok.save_pretrained("saved_mental_status_bert")

import pickle

pickle.dump(encode , open("encode.pkl" , "wb"))

In [ ]:
from google.colab import files
files.download("/content/saved_mental_status_bert/model.safetensors")


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("saved_mental_status_bert")

tok = AutoTokenizer.from_pretrained("saved_mental_status_bert")

encode = pickle.load(open("encode.pkl" , "rb"))

In [ ]:
def detection_text(text) :
    cleaned_text = preprocessing(text)

    inputs = tok(cleaned_text , return_tensors="pt" , padding=True , truncation=True , max_length=512)
    outputs = model(**inputs)
    logists = outputs.logits  # Purpose: Extracts the logits from the model outputs. Logits are the raw, unnormalized scores for each class before applying softmax.
    pred_classes = torch.argmax(logists , dim=1).item()
    return encode.inverse_transform([pred_classes])[0]
sample_texts = [
    "I feel perfectly fine today, nothing to worry about.",
    "I can't stop thinking about what will happen if things go wrong.",
    "Lately, I've been on a high, feeling like I can do anything!",
    "I'm so sad, I just can't seem to get out of bed anymore.",
    "I'm constantly thinking about how much better everyone else is doing than me.",
    "I don't think I can keep going, everything feels so hopeless.",
    "I had a really good day, spent some time with my friends.",
    "I'm overwhelmed by the idea that I might lose everything.",
    "I feel like nothing matters anymore, I just want to give up.",
    "I'm okay today, but sometimes I get really anxious for no reason."
]


for t in sample_texts :
    pred_classes = detection_text(t)
    print(f"Sentence: {t}\nPredicted class: ( {pred_classes} )\n")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("/content/saved_mental_status_bert")
tokenizer = AutoTokenizer.from_pretrained("/content/saved_mental_status_bert")

# Save both to Google Drive
model.save_pretrained('/content/drive/MyDrive/saved_mental_status_bert')
tokenizer.save_pretrained('/content/drive/MyDrive/saved_mental_status_bert')
